In [1]:
from preprocessing import get_model_dataset, create_train_test, min_max_scale, df_to_xy
from trainer import trainer
from lstm import create_model
import numpy as np
import matplotlib.pyplot as plt

In [43]:
import os
import pandas as pd
os.listdir("./data/rates")

pd = pd.read_csv("./data/options/spx_eod_202201.txt")
print(pd)

        [QUOTE_UNIXTIME]   [QUOTE_READTIME]  [QUOTE_DATE]  \
0             1641243600   2022-01-03 16:00    2022-01-03   
1             1641243600   2022-01-03 16:00    2022-01-03   
2             1641243600   2022-01-03 16:00    2022-01-03   
3             1641243600   2022-01-03 16:00    2022-01-03   
4             1641243600   2022-01-03 16:00    2022-01-03   
...                  ...                ...           ...   
188118        1643662800   2022-01-31 16:00    2022-01-31   
188119        1643662800   2022-01-31 16:00    2022-01-31   
188120        1643662800   2022-01-31 16:00    2022-01-31   
188121        1643662800   2022-01-31 16:00    2022-01-31   
188122        1643662800   2022-01-31 16:00    2022-01-31   

         [QUOTE_TIME_HOURS]   [UNDERLYING_LAST]  [EXPIRE_DATE]  \
0                      16.0             4795.57     2022-01-03   
1                      16.0             4795.57     2022-01-03   
2                      16.0             4795.57     2022-01-03   
3  

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (15,17,18,20,21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
path_opt = "./data/options/"
filenames_opt = ["spx_eod_" + str(year) + (str(month) if month >= 10 else "0"+str(month)) + ".txt" for year in range(2022, 2022) for month in range(1, 13)] + ["spx_eod_2022" + (str(month) if month >= 10 else "0" + str(month)) + ".txt" for month in range(1, 10)]
path_r = "./data/rates/"
filenames_r = ["yield-curve-rates-2022.csv", "yield-curve-rates-1990-2021.csv"]

df_read = get_model_dataset(path_opt, filenames_opt, path_r, filenames_r, True)
print(df_read)
df_read.info()

Empty DataFrame
Columns: [Quote_date, Expire_date, Underlying_last, Strike, Moneyness, Ask, Bid, Ttl, Volatility, R]
Index: []
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Quote_date       0 non-null      datetime64[ns]
 1   Expire_date      0 non-null      datetime64[ns]
 2   Underlying_last  0 non-null      float64       
 3   Strike           0 non-null      float64       
 4   Moneyness        0 non-null      float64       
 5   Ask              0 non-null      float64       
 6   Bid              0 non-null      float64       
 7   Ttl              0 non-null      int64         
 8   Volatility       0 non-null      float64       
 9   R                0 non-null      float64       
dtypes: datetime64[ns](2), float64(7), int64(1)
memory usage: 0.0 bytes


In [3]:
features = ["Moneyness", "Ttl", "R", "Volatility"]
seq_length = 5
num_features = 4

df_train_orginal, df_test_orginal = create_train_test(df_read, features,  "2022-09-01", 5)

train_x_org, train_y_org, = df_to_xy(df_train_orginal, num_features, seq_length)
test_x_org, test_y_org = df_to_xy(df_test_orginal, num_features, seq_length)

train_x_scaled, test_x_scaled = min_max_scale(train_x_org, test_x_org)
train_y_scaled, test_y_scaled = min_max_scale(train_y_org, test_y_org)

train_x_scaled = np.reshape(train_x_scaled, (len(train_x_scaled), seq_length, num_features))
test_x_scaled = np.reshape(test_x_scaled, (len(test_x_scaled), seq_length, num_features))

print(f"Train_x shape: {train_x_scaled.shape}, train_y shape: {train_y_scaled.shape}")
print(f"Test_x shape: {test_x_scaled.shape}, test_y shape: {test_y_scaled.shape}")

ValueError: Found array with 0 sample(s) (shape=(0, 20)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
from keras.callbacks import EarlyStopping
config = {
    "units": 100,
    "dropout": 0.1,
    "recurrent_dropout": 0.1,
    "learning_rate": 0.001,
    "layers": 5,
    "seq_length": seq_length,
    "num_features": num_features
}

def trainer(train_x, train_y, model):
    epochs = 10
    minibatch_size = 1024

    early_stopping = EarlyStopping(
        monitor='loss',
        mode='min',
        min_delta=1e-4,
        patience=10,
    )

    model.fit(
        train_x,
        train_y,
        batch_size = minibatch_size,
        epochs = epochs,
        
        callbacks = [early_stopping]
    )

model = create_model(config)

trainer(train_x_scaled, train_y_scaled, model)

Epoch 1/10
508/508 [==============================] - 152s 282ms/step - loss: 0.0075 - accuracy: 0.7412
Epoch 2/10
508/508 [==============================] - 157s 309ms/step - loss: 0.0025 - accuracy: 0.7574
Epoch 3/10
508/508 [==============================] - 171s 337ms/step - loss: 0.0017 - accuracy: 0.7521
Epoch 4/10
508/508 [==============================] - 174s 343ms/step - loss: 0.0011 - accuracy: 0.7504
Epoch 5/10
508/508 [==============================] - 179s 352ms/step - loss: 8.7219e-04 - accuracy: 0.7386
Epoch 6/10
508/508 [==============================] - 176s 347ms/step - loss: 6.7856e-04 - accuracy: 0.6955
Epoch 7/10
508/508 [==============================] - 185s 365ms/step - loss: 5.6360e-04 - accuracy: 0.6946
Epoch 8/10
508/508 [==============================] - 230s 454ms/step - loss: 5.3804e-04 - accuracy: 0.7058
Epoch 9/10
508/508 [==============================] - 248s 488ms/step - loss: 4.3530e-04 - accuracy: 0.7155
Epoch 10/10
508/508 [=======================

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def prediction(df_test, test_x, model, train_y_org):
    predictions = np.array(model(test_x))
    scaler = MinMaxScaler().fit(train_y_org)
    predictions = scaler.inverse_transform(predictions)
    bid, ask = predictions[:, :1], predictions[:, 1:]
    df_test["Prediction_bid"] = bid
    df_test["Prediction_ask"] = ask
    return df_test

df_test = prediction(df_test_orginal, test_x_scaled, model, train_y_org)

df_test.info()
print(df_test.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121672 entries, 1395271 to 1533711
Data columns (total 34 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Quote_date       121672 non-null  datetime64[ns]
 1   Expire_date      121672 non-null  datetime64[ns]
 2   Underlying_last  121672 non-null  float64       
 3   Strike           121672 non-null  float64       
 4   Moneyness        121672 non-null  float64       
 5   Ask              121672 non-null  float64       
 6   Bid              121672 non-null  float64       
 7   Ttl              121672 non-null  int64         
 8   Volatility       121672 non-null  float64       
 9   R                121672 non-null  float64       
 10  Moneyness-4      121672 non-null  float64       
 11  Ttl-4            121672 non-null  float64       
 12  R-4              121672 non-null  float64       
 13  Volatility-4     121672 non-null  float64       
 14  Moneyness-3  

In [ ]:
print(df_test.head(20)[["Bid", "Ask", "Prediction_bid", "Prediction_ask"]])
df_test.info()

            Bid     Ask  Prediction_bid  Prediction_ask
1395271  2997.2  3004.8     2966.038574     2921.669189
1395272  2797.3  2804.8     2743.442139     2708.014893
1395273  2597.1  2604.6     2526.692627     2497.127441
1395274  2397.1  2405.4     2311.479736     2285.217529
1395275  2197.6  2205.4     2094.024414     2069.406738
1395276  1998.1  2004.9     1872.450684     1848.897949
1395277  1797.7  1805.5     1647.211060     1625.150513
1395278  1597.5  1605.0     1420.696899     1401.313965
1395279  1397.3  1405.0     1196.016235     1180.693481
1395280  1297.3  1305.0     1084.816772     1071.880371
1395281  1197.3  1204.8      974.020264      963.546387
1395282  1150.9  1155.4      918.587463      909.333984
1395283  1100.9  1105.4      863.031799      854.973938
1395284  1051.0  1055.4      807.306580      800.409729
1395285  1000.4  1005.4      751.412109      745.633728
1395286   950.4   955.4      695.412231      690.703613
1395287   900.4   906.1      639.446594      635